<a href="https://colab.research.google.com/github/deep1185/ga-learner-dsmp-repo/blob/master/Hummingbird_Notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hummingbird-ml[extra]

     |████████████████████████████████| 184kB 6.8MB/s 
     |████████████████████████████████| 804.1MB 23kB/s 
     |████████████████████████████████| 71kB 62kB/s 
     |████████████████████████████████| 12.2MB 259kB/s 
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/drive',force_remount=True)

Mounted at /drive


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from hummingbird.ml import convert

In [ ]:
# We are going to use the breast cancer dataset from scikit-learn for this example.
X, y = load_breast_cancer(return_X_y=True)
nrows=15000
X = X[0:nrows]
y = y[0:nrows]

In [ ]:
# Create and train a random forest model.
model = RandomForestClassifier(n_estimators=10, max_depth=10)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
%%timeit -r 3

# Time for scikit-learn.
model.predict(X)

1000 loops, best of 3: 1.47 ms per loop


In [ ]:
model = convert(model, 'torch', extra_config={"tree_implementation":"gemm"})

In [ ]:
%%timeit -r 3

# Time for HB.
model.predict(X)

The slowest run took 5.69 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.28 ms per loop


In [ ]:
!nvidia-smi

Fri Jun  4 18:00:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
model.to('cuda')

In [ ]:
%%timeit -r 3

# Time for HB GPU.
model.predict(X)

The slowest run took 9.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 407 µs per loop
